---
date: '2025-05-07T18:39:45+08:00'
title: 'Remove Unused Confs'
---

In [ ]:
from surun_tools.glob1 import glob_files
glob_files("*.vcxproj")

[WindowsPath('C:/src/surun/src/PC/InstallSuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRunExt.vcxproj'),
 WindowsPath('C:/src/surun/tests/TestScreenshot/TestScreenshot.vcxproj'),
 WindowsPath('C:/src/surun/tests/TestScreenshotSuRun/TestScreenshotSuRun.vcxproj')]

In [10]:
proj_files = list(filter(lambda x: "test" not in str(x), glob_files("*.vcxproj")))
proj_files

[WindowsPath('C:/src/surun/src/PC/InstallSuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRunExt.vcxproj')]

In [11]:
ext_proj = glob_files("*.vcxproj")[2]

In [12]:
print(ext_proj)

C:\src\surun\src\PC\SuRunExt.vcxproj


In [4]:
from lxml import etree

In [13]:
def is_correct_conf(s: str) -> bool:
    if "x64 Unicode Debug|x64" in s:
        return True
    elif "SuRun32 Unicode Debug|Win32" in s:
        return True
    return False

In [18]:
def save():
    global tree
    tree.write(ext_proj, encoding="utf-8", xml_declaration=True)
    from pathlib import Path

    content = Path(ext_proj).read_text(encoding="utf8")
    content = content.replace("ns0:", "").replace(":ns0", "").replace('/>', ' />')
    Path(ext_proj).write_text(content, encoding="utf8")

In [19]:
ns = {"default": "http://schemas.microsoft.com/developer/msbuild/2003"}


parser = etree.XMLParser(remove_comments=False)
tree = etree.parse(ext_proj, parser)
root = tree.getroot()

for i in root.findall('default:ItemGroup/default:ProjectConfiguration', ns):
    if not is_correct_conf(i.attrib["Include"]):
        i.getparent().remove(i)

save()


for sln files:

remove these:
```
GlobalSection(SolutionConfigurationPlatforms) = preSolution
		Debug|Win32 = Debug|Win32 <-
		Debug|x64 = Debug|x64     <-
...
EndGlobalSection
```

for this file we can see that only conf contains | symbol. we can use that.

In [22]:
from pathlib import Path
sln:Path = glob_files("*.sln")[0]
print(sln)

C:\src\surun\src\PC\SuRun.sln


In [25]:
content = sln.read_text(encoding="utf8")
text = []
for i in content.split("\n"):
    if "|" in i:
        if not is_correct_conf(i):
            continue
        else:
            text.append(i)
    else:
        text.append(i)
sln.write_text("\n".join(text), encoding="utf8")

3861